# Setup
## Import Statements 
The import statements and setup function required to run this program are located below.

In [4]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Javascript, display
import ipython_blocking

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=False)
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## User Input
Below, the user should input the following information:

* Start Date - the date at which you would like to start the stock analysis
* End Date - the date at which you would like to end the stock analysis
* Stock Ticker - the symbol used to identify a stock or cryptocurrency (e.g. TSLA, BRK-B, BTC-USD)

Only the stock ticker is a required field.

The code will resume once the Start button is clicked.

In [15]:
start = widgets.DatePicker(description = 'Start Date')
end = widgets.DatePicker(description = 'End Date')
ticker = widgets.Text(description = "Stock Ticker")
button = widgets.Button(description = 'Start')
box = widgets.VBox(children = [start, end, ticker, button])
box
!pip install IPython

In [ ]:
%blockrun button

# Data
## Read Data
The code below uses pandas-datareader to get information about the selected stock from Yahoo Finance.

In [ ]:
stock = web.DataReader(ticker.value, 'yahoo', start.value, end.value).reset_index()
stock = round(stock, 2)
stock

## Calculate Percent Change
The code below calculates the percent change in the stock price each day.

In [ ]:
stock['Pct Change'] = stock['Close'].pct_change() * 100

## Calculate Moving Averages
The code below calculates the average stock price over the past 200 days (MA200) and past 50 days (MA50).

In [ ]:
for i in {200, 50}:
    stock["MA{}".format(i)] = stock['Close'].rolling(window = i).mean()

## Calculate Relative Strength Index (RSI)
<span class="mark">TODO: Explain</span>

In [ ]:
stock['Change'] = stock['Close'].diff()

stock['Up'] = stock['Change'].copy()
stock.loc[stock['Change'] <= 0.0, 'Up'] = 0.0

stock['Down'] = stock['Change'].copy()
stock.loc[stock['Change'] >= 0.0, 'Down'] = 0.0
stock['Down'] = abs(stock['Down'])

stock['RS'] = stock['Up'].rolling(window = 14).mean() / stock['Down'].rolling(window = 14).mean()
stock['RSI'] = 100 - (100 / (1 + stock['RS']))

## Display Data/Calculations
The code below displays the retrieved stock data and all of the calculations done.

In [ ]:
stock = round(stock, 2)
stock

# Graphs
## Candlestick
The interactive candlestick graph below shows the daily variation in stock price. The boxes represent the spread between opening and closing prices, while the lines represent the spread between the low and high prices for each day.

In [ ]:
fig = go.Figure(data = [go.Candlestick(x = stock['Date'],
               open = stock['Open'],
               high = stock['High'],
               low = stock['Low'],
               close = stock['Close'])])
fig.update_layout(title_text = 'Candlestick', 
                  xaxis_title = 'Date',
                  yaxis_title = 'Price',
                  xaxis_rangeslider_visible = True)
fig.show()

## Moving Average Convergence Divergence (MACD)
The interactive line graph below shows the closing price, the 200-day moving average, and the 50-day moving average for the stock. Moving averages show what is hapenning to the price of a stock (on average) over time:
* A buy signal occurs when the 50-day moving average crosses above the 200-day moving average.
* A sell signal occurs when the 50-day moving average drops below the 200-day moving average.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = stock['Date'], y = stock['Close'], name = 'Closing Price'))
fig.add_trace(go.Scatter(x = stock['Date'], y = stock['MA200'], name = '200-Day Moving Average'))
fig.add_trace(go.Scatter(x = stock['Date'], y = stock['MA50'], name = '50-Day Moving Average'))
fig.update_layout(title_text = 'Moving Average Convergence Divergence (MACD)', 
                  xaxis_title = 'Date',
                  yaxis_title = 'Price',
                  xaxis_rangeslider_visible = True)
fig.show()

The code below uses moving averages to indicate that the stock should potentially either be bought or sold (at the end date specified above):
* If the 50-day moving average is greater than the 200-day moving average, it tells you to buy.
* Otherwise, it tells you to sell.

In [ ]:
ma200 = stock.iloc[-1]['MA200']
ma50 = stock.iloc[-1]['MA50']
if ma50 > ma200:
    print('Buy Candidate')
else:
    print('Sell Candidate')

## Relative Strength Index (RSI)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = stock['Date'], y = stock['Close'], name = 'Closing Price'))
fig.add_trace(go.Scatter(x = stock['Date'], y = stock['RSI'], name = 'Relative Strength Index'))
fig.update_layout(title_text = 'Relative Strength Index (RSI)', 
                  xaxis_title = 'Date',
                  yaxis_title = 'Price',
                  xaxis_rangeslider_visible = True)
fig.show()